# Aterrizando las medidas

## Algunas medidas ##

A continuación se muestran algunas medias que pueden ser de utilidad en la identificacion de ataques DDoS:


| Caracteristicas        | Tipos de ataque         | 
| ------------- | ------------- | 
| Packet rate   | Flooding DDoS | 
| Percentage of ICMP packet     | icmp flooding      | 
| Percentage of TCP packet      |  tcp flooding      |   
| Percentage of UDP packet      | udp flooding | 
| Percentage of SYN/ACK/RST     | DDoS      | 
| Distribution of IP address    | DDoS | 
| Interval of arriving packet   | DDoS    | 
| Duration of each flow/session | SYN flooding      |   
| Packets/bytes per flow Flooding  | DDoS | 
| Percentage of single flow     | DDoS      | 
 


### ICMP flood ###


![wireshark_icmp_flood](wireshark_icmp_flood.png)


**Graficas I/O**

A continuación se muestra la tasa de paquetes para los protocolos icmp e icmp6 (aunque este ultimo no aparece). Se usan las siguientes caracteristicas:
* **Filtro**: ``icmp, icmp6``
* **Eje Y**: Packets/tick y escala lineal

![packet_rate](icmp_flood_packets_seg.png)

Como se puede ver la tasa de paquetes por segundo es alta.

**Summary**

La siguiente figura muestra un resumen asociado a la traza:

![summary](summary_icmp_flood.png)

**Estadisticas IPv4**

Siguiendo **Statistics > IPv4 > IP Protocol types** y agregando como filtro ``icmp`` el resultado se muestra a continuación:

![(ipv4_icmp_stat](ipv4_icmp_stat.png)



**Estadisticas asociadas a la longitud de los paquetes**

En el archivo [packet_length.csv](packet_length.csv) se encuentran estas tambien se despliegan en la siguiente figura:

![packet_len](packet_len.png)

Analizando todo lo anterior y mirando la tabla inicial para el caso de los paquetes icmp, se puede notar que para la traza el **Percentage of ICMP packet = (ICMP packets/Total packets)*100 = (961180/961180)*100 = 100%**

### SYN flood ###

En un escenarion en el cual se esta dando un SYN flood attack lo que pasa es lo siguiente:
* El cliente esta envianfo SYN muy rapido, ademas cuando el cliente recibe el SYN-ACK este no responde con el ACK final.
* Otro caso se da cuando cuando el cluente esta enviando muy rapido SYN y bloqueando el SYN-ACK del servidor, o tambien cuando el cliente esta enviando muy rapido SYN desde una direccion IP spoofed (falsificada) de manera que el SYN-ACK es enviado a un host desconocido que virtualmente no existe.

Para todos estos escenarios, los TCP/IP stack file descriptors son consumidos causando que el servidor (slow down) se sobrecargue y finalmente colapse

Una gradica tipica se muestra a continuación y en esta:
1. El conteo de paquetes SYN es muy alto.
2. El conteo de ACK es la mitad o incluso mucho menor que la de los paquetes SYN.
3. Como la conexión TCP nunca se cierra no hay conteo de paquetes en los que tcp.flags.fin este en 1.
4. Como la coneción TCP nunca intercambia algun dato no hay contro para tcp.flags.push 

Los filtros aplicados en la grafica son:
1. ``tcp.flags.syn == 1``
2. ``tcp.flags.ack == 1``
3. ``tcp.flags.fin == 1``
4. ``tcp.flags.push == 1``

![io_synflood](io_syn_flood.png) 

A continuación se muestra un resumen asociado al total de paquetes de la traza y a la regla de display ``tcp.flags.syn == 1`` 

![syn_flag](sum_syn_flag.png) 

A continuación se muestra un resumen asociado al total de paquetes de la traza y a la regla de display ``tcp.flags.ack == 1`` 

![ack_flag](sum_ack_flag.png) 

**Conversaciones**

Siguiendo la opcion **Statistics > conversations > TIPO DE CONVERSACION** se tiene para el caso:

* TCP: 539298
* tcp.flags.syn == 1: 539298  
* tcp.flags.ack == 1: 116

La verdad estoy perdido con el resultado anterior.

**End point List**

Siguiendo la opcion **Statistics > End point List > TCP**:

Hay en todal: 65537 endpoints (conexiones).

Siguiendo la opcion  **Statistics > End point List > IP**

Hay 3 conversaciones

**IPV4 statistics**

Siguiendo la opcion **Statistics > IPV4 statistics > Destination and ports** (Para el caso dado no se aplico filtro)

![ipv4_statistics](ipv4_statistics.png)


**Realizando conteos**

**SUM(*)**: Este calculo suma el valor de un campo:
* **Caso 1**: Graficar la cantidad de datos TCP (no incluye datalink, IP o TCP headers). Para ello usar en la traza el valor ``tcp.len``. Todo queda asi entonces para este ejemplo

  * **Grafica 1**:
    * **Filtro**: ip.src == 192.168.0.100; **Funcion**: SUM(*); **Valor**: tcp.len
  * **Grafica 2**:
    * **Filtro**: ip.dst == 192.168.0.100; **Funcion**: SUM(*); **Valor**: tcp.len

![syn_sum](syn_sum.png)

* **Caso 2**: Definir tcp.seq para graficar el numero de secuencia como si incrementara

  * **Grafica**:
    * **Filtro**: ---; **Funcion**: SUM(*); **Valor**: tcp.seq

![syn_seq](syn_seq.png)

**COUNTE(*)**: Cuenta la ocurrencia de una caracteristica. Es muy util cuando se grafica TCP analisis flags tales ``tcp.analysis.retransmission`` o ``tcp.analysis.duplicate_ack`` como:

* **Grafica**:
    * **Filtro**: ---; **Funcion**: COUNT(*); **Valor**: tcp.analysis.duplicate_ack
    * **Filtro**: ---; **Funcion**: COUNT(*); **Valor**: tcp.analysis.lost_segment
    * **Filtro**: ---; **Funcion**: COUNT(*); **Valor**: tcp.analysis.retransmission

![syn_count](syn_count.png)


## Parentesis - Caso a parte:  Graph and Analyze High TCP Delta Times ##

Para el caso se usa la función ``tcp.time_delta`` 

Seleccionando las caracteristicas avanzadas para la grafica se tiene:
* **Grafica roja**: MIN(*) y tcp.time_delta
* **Grafica verde**: AVG(*) y tcp.time_delta
* **Grafica roja**: MAX(*) y tcp.time_delta

![deltas](deltas.png)

La grafica tcp.time.time_delta puede ser empleada para detectar altos tiempos de respuesta para cualquier aplicacion basada en TCP.

**Interval arriving packet**:

* https://www.wireshark.org/lists/wireshark-users/200701/msg00120.html
* https://osqa-ask.wireshark.org/questions/13441/plot-arrival-time-of-udp-packets
* https://osqa-ask.wireshark.org/questions/30812/how-to-filter-inter-packet-arrival-time

* https://osqa-ask.wireshark.org/questions/53373/rtp-time-delta-from-previous-packet
* https://www.wireshark.org/docs/wsug_html_chunked/ChUsePacketListPaneSection.html
* https://osqa-ask.wireshark.org/questions/14995/get-packet-from-wireshark-from-cet-time-1600-to-1700


flow duration
* https://osqa-ask.wireshark.org/questions/26387/checking-the-flow-duration-in-wireshark
* https://osqa-ask.wireshark.org/questions/59975/print-duration-of-each-tcp-flows
* https://osqa-ask.wireshark.org/questions/27475/filter-on-tcp-stream-duration

**Pendiente** 

¿Como se maneja en un IDS este tipo de estadisticas?

https://github.com/eldondev/Snort/blob/master/rules/ddos.rules
https://serverfault.com/questions/178437/snort-rules-for-syn-flood-ddos
https://stackoverflow.com/questions/25825427/how-to-know-if-snort-detects-syn-flood-attacks-since-snort-alert-is-not-logging

etc...

https://lists.emergingthreats.net/pipermail/emerging-sigs/2015-January/025526.html
https://lists.openinfosecfoundation.org/pipermail/oisf-users/2014-February/003353.html


https://wiki.wireshark.org/Lua/Examples
https://www.caida.org/~emile/data_proc/lua_wireshark.html
https://softwarehealthclub.wordpress.com/ip/lua-script-which-generates-tcp-stats-and-outputs-to-csv/
https://github.com/gaddman/wireshark-tcpextend


### Enlaces ###

https://github.com/moranzcw/Computer-Networking-A-Top-Down-Approach-NOTES    